In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install pandas

In [ ]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains 
import re

## Сбор данных

In [ ]:
driver = webdriver.Chrome()
driver.get('https://decrypt.co/news-explorer')
#html = driver.page_source
#soup = BeautifulSoup(html)
data = []


while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    #for a in soup.find_all('a', href=True):
        #if len(re.findall(r"(\d+\/)(\S*)",a['href'])) or len(re.findall(r"(resources\/)(\S*)",a['href'])):
            #print("Found the URL:", a['href'])
            #pass
    
    containers = soup.find_all("div",attrs={"class":"flex gap-2 flex-col py-5 px-4 pb-3 md:p-6 md:pb-4 border-[1px] border-solid border-b-0 undefined relative"})
    for i, d in enumerate(containers):
        header = d.find("h4", {"class": "font-akzidenz-grotesk font-medium text-base md:text-xl degen-alley-dark:text-white"})
        if header is None:
            header = "NOT FOUND"
        else:
            header = header.text.strip()

        timedata = d.find("div", {"class": "text-sm text-neutral-700 degen-alley-dark:text-neutral-200"})
        if timedata is None:
            timedata = "NOT FOUND"
        else:
            timedata = timedata.text.strip()

        ref = d.find("a", {"class": "py-2 text-center space-x-2 font-akzidenz-grotesk font-medium text-sm leading-none"})
        if ref is None:
            ref = "NOT FOUND"
        else:
            ref = ref['href'].strip()

        #if (header, timedata, ref) in data:
        #    continue
        if (header, timedata, ref) in data:
            continue
        
        data.append((header, timedata, ref))
        print(f"Found header: {header}, date: {timedata}, ref: {ref}")
        #time_data = soup.find_all("div",attrs={"class":"post-loop__date"})   
    #print(len(data))
    button = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[1]/div/main/div[4]/div/div[1]/div/button')
    location = button.location
    size = button.size
    w, h = size['width'], size['height']
    driver.execute_script(f"window.scrollTo(0, {h});")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(1)


## Обработка данных

In [ ]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link'])
df = df.drop_duplicates()
df.head()

In [ ]:
df.to_csv("../data/decrypt_no_index.csv", index=False)

In [ ]:
df.tail()
#df = df.drop(columns='Unnamed: 0')
df = df.rename(columns={"href": "link"})
df.tail()

## Чтение данных из файла

In [ ]:
df = pd.read_csv("../data/decrypt_no_index.csv")

# Маркировка по годам

In [ ]:
print(list(df['date']))

In [ ]:
current_year = 2025
reached_30dec = False

for i, dtime in enumerate(list(df['date'])):
    year_counted = current_year
    cdate, ctime = [s.strip() for s in dtime.split(',')]
    #print(cdate, ctime)
    
    if cdate == "Dec 30":
        reached_30dec = True
        year_counted -= 1
    elif reached_30dec:
        current_year -= 1
        reached_30dec = False
    
    df.loc[i, 'date'] = f"{cdate}, {ctime}, {year_counted}"
    
df

In [ ]:
df.to_csv("../data/decrypt_with_years.csv", index=False)
# xd